# SMS Spam Prediction using Deep Learning

In [1]:
#import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Abid\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
#loading the data
df = pd.read_csv('spam.csv',usecols = [0,1],names = ['Spam','SMS'],skiprows = 1)
df.head(3)

,Spam,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...


In [3]:
#Check whether there are any null values
df.isna().sum()

Spam    0
SMS     0
dtype: int64

In [4]:
df.describe()

,Spam,SMS
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


Note:
    
    There are 5572 records out of which 5169 are unique

In [5]:
#Storing all the stop words in a variable
stopwords_list = stopwords.words('english') 

In [6]:
#Initializing PorterStemmer and WordNetLemmatizer
stemmer = PorterStemmer()
Lemmatizer = WordNetLemmatizer()

In [8]:
#All the processed sentences will be stored in corpus list
corpus = []
for i in range(df.shape[0]):
    sent = re.sub('[^a-zA-Z]',' ',df['SMS'][i]) #Removes all Special Characters and punctuations
    sent = re.sub('\s+[a-zA-Z]\s+',' ',sent) #Removes all single characters
    sent = re.sub('\s+',' ',sent) #Replace multiple spaces with a space
    sent = sent.lower() #Convert all words into lower case
    words = nltk.word_tokenize(sent) #Tokenization of sentence into words
    words = [Lemmatizer.lemmatize(word) for word in words if word not in stopwords_list] #Applying Lemmatization
    processed_sent = ' '.join(words)
    corpus.append(processed_sent)
    

In [9]:
#Top 10 processed rows in corpus
corpus[:10]

['go jurong point crazy available bugis great world la buffet cine got amore wat',
 'ok lar joking wif oni',
 'free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate c apply',
 'u dun say early hor c already say',
 'nah think go usf life around though',
 'freemsg hey darling week word back like fun still tb ok xxx std chgs send rcv',
 'even brother like speak treat like aid patent',
 'per request melle melle oru minnaminunginte nurungu vettam set callertune caller press copy friend callertune',
 'winner valued network customer selected receivea prize reward claim call claim code kl valid hour',
 'mobile month r entitled update latest colour mobile camera free call mobile update co free']

In [10]:
#Imputing Spam with 1 and Ham with 0
sms_dict = {'ham': 0, 'spam': 1}
df['Spam'] = df['Spam'].map(sms_dict)

In [11]:
#Apply Bag of Words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features= 5000)
X = cv.fit_transform(corpus).toarray()

In [12]:
y = df[['Spam']]

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.25,random_state = 0)

### NAIVE BAYES

In [14]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(x_train,y_train)

C:\Users\Abid\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


MultinomialNB()

In [15]:
mnb.score(x_test,y_test)

0.9820531227566404

In [22]:
from sklearn.model_selection import cross_val_score, KFold
kfold = KFold(n_splits = 5)

In [23]:
cross_validation = cross_val_score(mnb,x_train,y_train,scoring = 'accuracy',cv = kfold )

C:\Users\Abid\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Abid\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Abid\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Abid\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for

In [26]:
print('Cross Validation Score is: {}'.format(cross_validation.mean()))

Cross Validation Score is: 0.9806188579778243


### ANN MODEL

With our data loaded and preprocessed, we’re now well prepared to use neural network architecture to classify the text message.

In [16]:
import tensorflow as tf

In [17]:
from tensorflow import keras
from tensorflow.keras.layers import Dense,ReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD

In [18]:
#Initialize Model
model = Sequential()

#Input Layer
model.add(Dense(7,activation = 'relu',kernel_initializer = 'uniform', input_dim = 5000))

#Hidden Layer
model.add(Dense(9,activation = 'relu',kernel_initializer = 'uniform'))

#Output Layer
model.add(Dense(1,activation = 'sigmoid',kernel_initializer = 'uniform'))


In [19]:
#Compilation
opt = SGD(learning_rate = 0.1,momentum = 0.5)
model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [20]:
#Accuracy on Train Data
model.fit(x_train,y_train,batch_size = 4,epochs = 5,validation_data = (x_test,y_test))

Epoch 1/5
1045/1045 [==============================] - 2s 2ms/step - loss: 0.2923 - accuracy: 0.9031 - val_loss: 0.0947 - val_accuracy: 0.9698
Epoch 2/5
1045/1045 [==============================] - 2s 1ms/step - loss: 0.0686 - accuracy: 0.9825 - val_loss: 0.0654 - val_accuracy: 0.9792
Epoch 3/5
1045/1045 [==============================] - 1s 1ms/step - loss: 0.0317 - accuracy: 0.9921 - val_loss: 0.0851 - val_accuracy: 0.9742
Epoch 4/5
1045/1045 [==============================] - 1s 1ms/step - loss: 0.0193 - accuracy: 0.9957 - val_loss: 0.0672 - val_accuracy: 0.9835
Epoch 5/5
1045/1045 [==============================] - 1s 1ms/step - loss: 0.0081 - accuracy: 0.9974 - val_loss: 0.0848 - val_accuracy: 0.9813


In [28]:
#Accuracy on Test data
model.evaluate(x_test,y_test,batch_size = 4)

349/349 [==============================] - 0s 821us/step - loss: 0.0848 - accuracy: 0.9813


[0.08480959385633469, 0.9813352227210999]